In [1]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [2]:
with open('vicRoads_faq.json','rt') as f:
    raw_documents = json.load(f)

In [3]:
raw_documents[1]

{'Section': 'Digital Driver Licence',
 'Question': "I'm concerned about safety and privacy as someone who has experienced family violence. How can I protect my personal details?",
 'Answer': 'Customers impacted by family violence can contact us through our dedicated phone line or email:\nPhone: 1300 031 292 (business hours)\nEmail: assistance@roads.vic.gov.au\nYou can leave your contact number if the telephone line is unavailable or unattended. Our service also supports agencies acting on behalf of affected persons. \n'}

In [4]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# checking the embedding model
embedding_model.encode("hello world")

array([ 2.62497254e-02,  1.33956019e-02, -4.53314278e-03, -2.17914544e-02,
        5.45518659e-02, -4.96648299e-03,  6.65560272e-03,  3.06262691e-02,
       -5.76278474e-03, -4.56204684e-03, -3.31329461e-03, -4.84962501e-02,
       -1.13640223e-02,  3.50774229e-02,  9.30946916e-02, -8.66874084e-02,
        5.10865487e-02,  9.88610461e-03, -6.35692477e-02, -8.55022110e-03,
        7.05439178e-03, -3.86239309e-03,  2.47442871e-02,  4.28849421e-02,
        3.50941271e-02, -2.98482478e-02,  1.02525903e-02,  2.23449301e-02,
        2.08899938e-02,  9.49220732e-03, -3.30444127e-02, -1.22841550e-02,
        5.35289273e-02,  2.54291948e-02,  2.02217666e-06, -3.41910198e-02,
        9.61000286e-03, -1.64845269e-02,  5.60948672e-03, -4.25002817e-03,
       -2.28012074e-02,  4.03546654e-02,  3.05205584e-03,  3.13725919e-02,
       -1.08123543e-02, -3.55707854e-02,  2.22929213e-02,  1.68712495e-03,
        2.07724981e-03,  2.31162217e-02,  6.88587269e-03, -6.83094654e-03,
       -4.87613045e-02, -

In [6]:
combined_documents = []

for document in raw_documents:
    text = document['Section']+"\n"+ document['Question']+"\n"+ document['Answer']
    combined_documents.append(text)

In [7]:
combined_documents[1]

"Digital Driver Licence\nI'm concerned about safety and privacy as someone who has experienced family violence. How can I protect my personal details?\nCustomers impacted by family violence can contact us through our dedicated phone line or email:\nPhone: 1300 031 292 (business hours)\nEmail: assistance@roads.vic.gov.au\nYou can leave your contact number if the telephone line is unavailable or unattended. Our service also supports agencies acting on behalf of affected persons. \n"

In [8]:
embeddings = []

for doc in combined_documents:
    embeddings.append(embedding_model.encode(doc))

In [9]:
embeddings = np.array(embeddings).astype('float32')

In [10]:
index = faiss.IndexFlatL2(768) 

In [11]:
index.add(embeddings)

In [12]:
embeddings.shape

(157, 768)

In [13]:
# saving index
faiss.write_index(index, "vr_faq_index.idx")